In [1]:

import sys
!{sys.executable} -m pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import time

# Function to scroll to the bottom
def scroll_to_bottom(driver):
    old_position = driver.execute_script("return window.pageYOffset;")
    while True:
        # Scroll to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait for page to load (adjust time as necessary)
        time.sleep(1)
        # Calculate new scroll position and compare with last scroll position
        new_position = driver.execute_script("return window.pageYOffset;")
        if new_position == old_position:
            break
        old_position = new_position

In [9]:
%pip install selenium
%pip install bs4
%pip install supabase
%pip install python-dotenv

from bs4 import BeautifulSoup
from selenium import webdriver
from supabase import create_client, Client
from collections import defaultdict
from dotenv import load_dotenv

import time
import os

load_dotenv()

url: str = os.getenv("SUPABASE_URL")
key: str = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(url, key)

options = webdriver.ChromeOptions()
options.add_argument('--headless')

driver = webdriver.Chrome(options=options)

x_descriptions, y_descriptions = set(), set()
x_company_dict, y_company_dict = defaultdict(list), defaultdict(list)

batches = [season + year for year in [str(i).zfill(2) for i in range(5, 24)] for season in ["S", "W"]]

for batch in batches:
    
    print(batch)
    driver.get(f"https://www.ycombinator.com/companies?batch={batch}")

    # Scroll function here
    scroll_to_bottom(driver)

    # Get the HTML source code of the page
    html = driver.page_source

    # Parse the HTML using Beautiful Soup
    soup = BeautifulSoup(html, "html.parser")
    
    # Get all company names and descriptions
    raw_companies = soup.find_all("span", class_="_coName_86jzd_453")
    raw_descriptions = soup.find_all("span", class_="_coDescription_86jzd_478")
    print("num companies")
    print(len(raw_companies))
    
    # Convert them to lists of strings
    descriptions = [str(x.text) for x in list(raw_descriptions)]
    companies = [str(x.text) for x in list(raw_companies)]
    
    filtered_descriptions = []
    filtered_companies = []
    for description, company in zip(descriptions, companies):
        if " for " in description:
            filtered_descriptions.append(description)
            filtered_companies.append(company)
    
    # Split all descriptions at the "for"
    for idx, filtered_description in enumerate(filtered_descriptions):
        split_description = filtered_description.split(" for ")
        x_description, y_description = split_description[0], split_description[1]
        x_descriptions.add(x_description)
        y_descriptions.add(y_description)
        company = filtered_companies[idx]
        x_company_dict[x_description].append(company)
        y_company_dict[y_description].append(company)
        original_company_data = {
            "company": company,
            "x_description": x_description,
            "y_description": y_description
        }
        response = supabase.table("original_companies").insert(original_company_data).execute()
    
for idx_x, x_description in enumerate(x_descriptions):
    for idx_y, y_description in enumerate(y_descriptions):
        if idx_x != idx_y:
            x_company = x_company_dict[x_description]
            y_company = y_company_dict[y_description]
            combination_data = {
                "company_x": x_company,
                "company_y": y_company,
                "x_description": x_description,
                "y_description": y_description,
            }
            response = supabase.table("combinations").insert(combination_data).execute()

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
S05
num companies
8
W05
num companies
0
S06
num companies
11
W06
num companies
7
S07
num companies
19
W07
num companies
13
S08
num companies
22
W08
num companies
21
S09
num companies
26
W09
num companies
16
S10
num companies
36
W10
num companies
27
S11
num companies
60
W11
num companies
45
S12
num companies
82
W12
num companies
66
S13
num companies
52
W13
num companies
46
S14
num companies
80
W14
num companies
74
S15
num